# Pouso de meteoritos no planeta terra

- Iara Araujo
- Beatriz Barreto 
- Paloma Duarte

## Introdução

### Motivação e objetivo


## Dados utilizados

### Dataset meteorite_landings
- Dados coletados pela [meteorical society](https://meteoritical.org/), incluindo localizações, massas,composições e ano de queda 
para mais de 45.000 meteoritos que caíram no planeta terra.
- Variáveis desse dataset:
    - **name**: nome do meteorito(geralmente relacionado com a localização onde atingiu a terra).
    - **id** : identificador unico para o meteorito no dataset.
    - **nametype**: classificação do meteorito em **relict** ou **valid**
    - **mass** : a massa do meteorito dada em gramas.
    - **fell**: classificação do meteorito em:
        - _Fall_: se foi identificado durante sua queda.
        - _Fell_: se foi identificado após sua queda.
    - **year**: ano em que o meteorito foi identificado.
    - **reclat**: latitude da posição em que o meteoro foi encontrado.
    - **reclong**:longitude da posição em que o meteoro foi encontrado.
    - **GeoLocation**: junção da latitude e da longitude em uma variável, separadas por vírgula.    
- Observações adicionais
    - as variáveis 'reclat' e 'reclong' são "valores recomendados", isto é levemente alterados do valor inicialmente dado, isso se deve a uma reclassificação do meteorito ou alteração dos dados originais de sua coleta.
    - alguns dados estão incorretos ou com difícil visualização, por isso faremos a limpeza na fase de pré processamento.

In [7]:
import pandas as pd
import numpy as np
import math
from math import log
import seaborn as sns
from scipy.stats import linregress
import matplotlib.pyplot as plt

In [8]:
df1 = pd.read_csv('datasets/meteorite_landings.csv', on_bad_lines='warn')
df2 = pd.read_csv('datasets/meteorite_landings_on_earth.csv', on_bad_lines='warn')   #lendo os datasets
df3 = pd.read_csv('datasets/cneos_fireball_data.csv', on_bad_lines='warn')